In [3]:
config_dir = "../experiments"
exp_name = "201_unet_multi/all"

In [4]:
%cd /kaggle/working

import pickle
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
from hydra import compose, initialize
from omegaconf import OmegaConf

from utils.metric import score

with initialize(
    version_base=None, config_path=f"{config_dir}/{exp_name.split('/')[0]}"
):
    cfg = compose(
        config_name="config.yaml",
        overrides=[f"exp={exp_name.split('/')[-1]}"],
        return_hydra_config=True,
    )

# 定数定義
output_dir = Path(f"output/experiments/{exp_name}")
gcs_path = f"gs://{cfg.dir.gcs_bucket}/{cfg.dir.gcs_base_dir}/experiments/{exp_name}/"

# 結果などの読み込み
r2_score_dict = pickle.load(open(output_dir / "val2_r2_score_dict.pkl", "rb"))
print("r2: ", np.mean(list(r2_score_dict.values())))

/kaggle/working
r2:  0.7815333871933391


In [5]:
predict_df = pl.read_parquet(gcs_path + "val2_predict.parquet", retries=5)
label_df = pl.read_parquet(gcs_path + "val2_label.parquet", retries=5)
ss_df = pl.read_csv(
    "input/leap-atmospheric-physics-ai-climsim/sample_submission.csv", n_rows=1
)
weight_array = ss_df.select([x for x in ss_df.columns if x != "sample_id"]).to_numpy()[
    0
]

In [41]:
# 通常
preds = predict_df[:, 1:].to_numpy()
labels = label_df[:, 1:].to_numpy()
_predict_df = pd.DataFrame(
    preds * weight_array, columns=[i for i in range(preds.shape[1])]
).reset_index()
_label_df = pd.DataFrame(
    labels * weight_array, columns=[i for i in range(labels.shape[1])]
).reset_index()
r2_scores = score(_label_df, _predict_df, "index", multioutput="raw_values")

r2_score = float(np.array(r2_scores).mean())
print(f"{r2_score=}")

r2_score=0.7815333871933391


In [42]:
r2_scores[132:142]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [44]:
# 10個の後処理結果を0埋めに変換

preds = predict_df[:, 1:].to_numpy()
preds[:, 132:142] = 0
labels = label_df[:, 1:].to_numpy()
_predict_df = pd.DataFrame(
    preds * weight_array, columns=[i for i in range(preds.shape[1])]
).reset_index()
_label_df = pd.DataFrame(
    labels * weight_array, columns=[i for i in range(labels.shape[1])]
).reset_index()
r2_scores = score(_label_df, _predict_df, "index", multioutput="raw_values")

r2_score = float(np.array(r2_scores).mean())
print(f"{r2_score=}")

r2_score=0.7538482690839238


In [47]:
(1-r2_scores[132:142]).sum()/368

0.027685118109415333

In [48]:
(1-r2_scores[132:142]).sum()/368 + 0.7538482690839238

0.7815333871933391